In [1]:
import numpy as np 
from keras.models import * 
from keras.layers import * 
from keras.optimizers import * 
from keras.datasets import mnist 
import keras.backend as K 
import matplotlib.pyplot as plt 
import time

K.set_image_data_format('channels_last') 

Using TensorFlow backend.


In [2]:
class Gan: 
    def __init__(self, img_data): 
        img_size = img_data.shape[1] 
        channel = img_data.shape[3] if len(img_data.shape) >= 4 else 1 
        self.img_data = img_data 
        self.input_shape = (img_size, img_size, channel) 
        self.img_rows = img_size 
        self.img_cols = img_size 
        self.channel = channel 
        self.noise_size = 100 
        
        self.create_d() 
        self.create_g() 
        
        optimizer = Adam(lr=0.0008) 
        self.D.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
        
        optimizer = Adam(lr=0.0004) 
        self.D.trainable = False 
        self.AM = Sequential() 
        self.AM.add(self.G) 
        self.AM.add(self.D) 
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer) 
        print('AM')
        #self.AM.summary()
        
    def create_d(self): 
        self.D = Sequential() 
        depth = 64 
        dropout = 0.4 
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=self.input_shape, padding='same')) 
        self.D.add(LeakyReLU(alpha=0.2)) 
        self.D.add(Dropout(dropout)) 
        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same')) 
        self.D.add(LeakyReLU(alpha=0.2)) 
        self.D.add(Dropout(dropout)) 
        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same')) 
        self.D.add(LeakyReLU(alpha=0.2)) 
        self.D.add(Dropout(dropout)) 
        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same')) 
        self.D.add(LeakyReLU(alpha=0.2)) 
        self.D.add(Dropout(dropout)) 
        self.D.add(Flatten()) 
        self.D.add(Dense(1)) 
        self.D.add(Activation('sigmoid')) 
        print('Discriminator')
        #self.D.summary() 
        return self.D 
        
    def create_g(self): 
        self.G = Sequential() 
        dropout = 0.4 
        depth = 64+64+64+64 
        dim = 7 
        self.G.add(Dense(dim*dim*depth, input_dim=self.noise_size)) 
        self.G.add(BatchNormalization(momentum=0.9)) 
        self.G.add(Activation('relu')) 
        self.G.add(Reshape((dim, dim, depth))) 
        self.G.add(Dropout(dropout)) 
        self.G.add(UpSampling2D()) 
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same')) 
        self.G.add(BatchNormalization(momentum=0.9)) 
        self.G.add(Activation('relu')) 
        self.G.add(UpSampling2D()) 
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same')) 
        self.G.add(BatchNormalization(momentum=0.9)) 
        self.G.add(Activation('relu')) 
        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same')) 
        self.G.add(BatchNormalization(momentum=0.9)) 
        self.G.add(Activation('relu')) 
        self.G.add(Conv2DTranspose(1, 5, padding='same')) 
        self.G.add(Activation('sigmoid')) 
        print('Generator')
        #self.G.summary() 
        return self.G 
        
    def train(self, batch_size=100): 
        images_train = self.img_data[np.random.randint(0, self.img_data.shape[0], size=batch_size), :, :, :] 
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, self.noise_size]) 
        images_fake = self.G.predict(noise) 
 
        x = np.concatenate((images_train, images_fake)) 
        y = np.ones([2*batch_size, 1]) 
        y[batch_size:, :] = 0 
        self.D.trainable = True 
        d_loss = self.D.train_on_batch(x, y) 
        decision = self.D.predict(images_fake)

        y = np.ones([batch_size, 1]) 
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, self.noise_size]) 
        self.D.trainable = False 
        a_loss = self.AM.train_on_batch(noise, y) 

        return d_loss, a_loss, images_fake, decision


In [3]:
#x_train, x_test, y_train, y_test = np.load('binary_image_data.npy')
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1).astype('float32')
x_train = (x_train -127.5) / 127.5 
print(x_train.shape) 

# Init network 
gan = Gan(x_train) 

(60000, 28, 28, 1)
Discriminator
Generator
AM


In [ ]:
맨 처음 확인 
# generator = gan.create_g()
# noise = np.random.uniform(-1.0, 1.0, size=[1,100])
# generated_image = generator.predict(noise)
# plt.imshow(generated_image.reshape((28,28)), cmap='gray')

# discriminator = gan.create_d()
# decision = discriminator.predict(generated_image)
# print (decision)

In [4]:
# Some parameters. 
epochs = 2
sample_size = 10 
batch_size = 100
train_per_epoch = x_train.shape[0] // batch_size
history=[]
print(train_per_epoch)

600


In [12]:
start_time = time.time()
total_d_loss = 0.0 
total_a_loss = 0.0
total_decision = []

d_loss, a_loss, imgs, decision = gan.train(batch_size) 

t_decisi = 0.0
for i in range(len(decision)) :
    if decision[i] > 0 :
        t_decisi += decision[i] 
        #print(decision[i])
        #plt.imshow(imgs[i].reshape((gan.img_rows, gan.img_cols)))
        #plt.show()
    else :    
        print(decision[i])

ttt = len(t_decisi)
print(ttt)
tt_decisi = t_decisi / ttt
print(tt_decisi)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


TypeError: object of type 'float' has no len()

In [113]:
disei = gan.create_d()
y_= disei.predict(imgs[:10])
predicted = np.argmax(y_, axis=1)
print(predicted)
print(y_.shape)
print(y_)

#print(disei.evaluate(test_x, y_)[1])
print('정답률(Accuracy):', metrics.accuracy(test_x[:10], y_[0]))

Discriminator
[0 0 0 0 0 0 0 0 0 0]
(10, 1)
[[0.4978253 ]
 [0.49781877]
 [0.49782714]
 [0.49776918]
 [0.49781188]
 [0.49780712]
 [0.497805  ]
 [0.4977731 ]
 [0.49778843]
 [0.49783185]]
정답률(Accuracy): tf.Tensor(
[[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...

In [ ]:
test_x = x_test[ : 1000]
test_x = test_x.reshape((test_x.shape[0],) + (28, 28, 1)).astype('float32') 
test_x = test_x / 255.0 

print(test_x.shape) 
plt.imshow(test_x[1].reshape((28,28)))
plt.show()
plt.close()


In [ ]:
# 원본
# Some parameters. 
epochs = 2
sample_size = 10 
batch_size = 100
train_per_epoch = x_train.shape[0] // batch_size
history=[]


for epoch in range(0, epochs): 
    start_time = time.time()
    total_d_loss = 0.0 
    total_a_loss = 0.0
    
    for batch in range(0, train_per_epoch): 
        d_loss, a_loss, imgs = gan.train(batch_size) 
        total_d_loss += d_loss 
        total_a_loss += a_loss 
        
    total_d_loss /= train_per_epoch 
    total_a_loss /= train_per_epoch
    
    recode =(epoch, total_d_loss, total_a_loss)
    history.append(recode)

    print("Epoch : {}, elapsed : {}, D Loss: {}, AM Loss: {}".format(epoch + 1, time.time() - start_time, total_d_loss, total_a_loss)) 
    
    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1)) 
    
    for i in range(0, sample_size): 
        ax[i].set_axis_off() 
        ax[i].imshow(imgs[i].reshape((gan.img_rows, gan.img_cols)), interpolation='nearest')
    plt.show() 
    plt.close(fig)


In [ ]:
from pandas import Series, DataFrame 

df = DataFrame(history, columns=['epoch', 'total_d_loss', 'total_a_loss'])

df.plot(y=['d_loss', 'a_loss'])
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()